### Before running the following script, make sure to setup the server in local with following command:

bash setup.sh

or

uvicorn app.main:app --reload --port 8080

In [1]:
from app.config import get_settings
import requests

In [2]:
api_base_url = get_settings().BASE_URL
api_service_url = '/'.join([api_base_url, 'service'])
api_info_url = '/'.join([api_base_url, 'info'])

In [5]:
api_base_url

'http://localhost:8080'

### Call API to Create Shortened URL

In [4]:
# Data to send
data = {
    "long_url": "https://www.keysight.com/us/en/home.html",
    "expiration_minutes": 20  # URL will expire after 20 minutes
}

# Send the POST request
response = requests.post(api_service_url, json=data)

if response.status_code == 200:
    json_response = response.json() 
    print("Shortened URL: {:s}/{:s}, copy to browser for testing the link".format(api_base_url, json_response.get("short_url")))
else:
    print("Error:", response.status_code, response.text)

Shortened URL: http://localhost:8080/HVDW4, copy to browser for testing the link


In [13]:
response.json()

{'long_url': 'https://www.google.com',
 'expiration_minutes': 20,
 'expiration_time': '2024-09-27T02:30:04.797886',
 'creation_time': '2024-09-27T02:10:04.797886',
 'short_url': 'XL6MS',
 'is_active': True}

### Check Shortened URL Information

In [6]:
url = '{:s}/{:s}'.format(api_info_url, 'HVDW4')
# Send the get request
response = requests.get(url, json=data)

In [7]:
response.json()

{'long_url': 'https://www.keysight.com/us/en/home.html',
 'remaining_time': '00:17:13',
 'expiration_time': '2024-09-27T02:49:13.572413',
 'creation_time': '2024-09-27T02:29:13.572413',
 'short_url': 'HVDW4',
 'is_active': True}

### Add More Links

In [8]:
# Data to send
data = {
    "long_url": "https://www.google.com",
    "expiration_minutes": 1  # URL will expire after 1 minutes
}
# Send the POST request
response = requests.post(api_service_url, json=data)

if response.status_code == 200:
    json_response = response.json() 
    print("Shortened URL: {:s}/{:s}, copy to browser for testing the link".format(api_base_url, json_response.get("short_url")))
else:
    print("Error:", response.status_code, response.text)

Shortened URL: http://localhost:8080/5LT40, copy to browser for testing the link


In [9]:
# Data to send
data = {
    "long_url": "https://www.bing.com",
    "expiration_minutes": 2  # URL will expire after 2 minutes
}
# Send the POST request
response = requests.post(api_service_url, json=data)

if response.status_code == 200:
    json_response = response.json() 
    print("Shortened URL: {:s}/{:s}, copy to browser for testing the link".format(api_base_url, json_response.get("short_url")))
else:
    print("Error:", response.status_code, response.text)

Shortened URL: http://localhost:8080/DPJ4S, copy to browser for testing the link


In [10]:
# Data to send
data = {
    "long_url": "https://www.amazon.com",
    "expiration_minutes": 3  # URL will expire after 3 minutes
}
# Send the POST request
response = requests.post(api_service_url, json=data)

if response.status_code == 200:
    json_response = response.json() 
    print("Shortened URL: {:s}/{:s}, copy to browser for testing the link".format(api_base_url, json_response.get("short_url")))
else:
    print("Error:", response.status_code, response.text)

Shortened URL: http://localhost:8080/3XM3B, copy to browser for testing the link


In [11]:
# Data to send
data = {
    "long_url": "https://github.com/",
    "expiration_minutes": 4  # URL will expire after 4 minutes
}
# Send the POST request
response = requests.post(api_service_url, json=data)

if response.status_code == 200:
    json_response = response.json() 
    print("Shortened URL: {:s}/{:s}, copy to browser for testing the link".format(api_base_url, json_response.get("short_url")))
else:
    print("Error:", response.status_code, response.text)

Shortened URL: http://localhost:8080/XPHKC, copy to browser for testing the link


### Check URLs in Database And Their Expiration

In [16]:
from app.database import SessionLocal, URLMapping
import pandas as pd
import datetime

In [14]:
db = SessionLocal()
df = []

for db_url_mapping in db.query(URLMapping).all():
    d = {'id': db_url_mapping.id,
         'creation_time': db_url_mapping.creation_time,
         'expiration_time': db_url_mapping.expiration_time,
         'is_active': db_url_mapping.is_active,
         'short_url': db_url_mapping.short_url,
         'long_url': db_url_mapping.long_url}
    df.append(d)

In [18]:
pd.DataFrame(df)

,id,creation_time,expiration_time,is_active,short_url,long_url
0,1,2024-09-27 02:29:13.572413,2024-09-27 02:49:13.572413,True,HVDW4,https://www.keysight.com/us/en/home.html
1,2,2024-09-27 02:37:14.979259,2024-09-27 02:38:14.979259,True,5LT40,https://www.google.com
2,3,2024-09-27 02:37:15.532955,2024-09-27 02:39:15.532955,True,DPJ4S,https://www.bing.com
3,4,2024-09-27 02:37:16.090145,2024-09-27 02:40:16.090145,True,3XM3B,https://www.amazon.com
4,5,2024-09-27 02:37:16.634263,2024-09-27 02:41:16.634263,True,XPHKC,https://github.com/


In [19]:
current_time = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
current_time

datetime.datetime(2024, 9, 27, 2, 38, 4, 516309)

#### After 1 minute, the program automatically deativate expired item [google]

In [21]:
db = SessionLocal()
df = []

for db_url_mapping in db.query(URLMapping).all():
    d = {'id': db_url_mapping.id,
         'creation_time': db_url_mapping.creation_time,
         'expiration_time': db_url_mapping.expiration_time,
         'is_active': db_url_mapping.is_active,
         'short_url': db_url_mapping.short_url,
         'long_url': db_url_mapping.long_url}
    df.append(d)
db.close()
pd.DataFrame(df)

,id,creation_time,expiration_time,is_active,short_url,long_url
0,1,2024-09-27 02:29:13.572413,2024-09-27 02:49:13.572413,True,HVDW4,https://www.keysight.com/us/en/home.html
1,2,2024-09-27 02:37:14.979259,2024-09-27 02:38:14.979259,False,5LT40,https://www.google.com
2,3,2024-09-27 02:37:15.532955,2024-09-27 02:39:15.532955,True,DPJ4S,https://www.bing.com
3,4,2024-09-27 02:37:16.090145,2024-09-27 02:40:16.090145,True,3XM3B,https://www.amazon.com
4,5,2024-09-27 02:37:16.634263,2024-09-27 02:41:16.634263,True,XPHKC,https://github.com/


In [22]:
current_time = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
current_time

datetime.datetime(2024, 9, 27, 2, 39, 7, 909831)

#### After 1 minute, the program automatically deativate expired item [bing]

In [24]:
db = SessionLocal()
df = []

for db_url_mapping in db.query(URLMapping).all():
    d = {'id': db_url_mapping.id,
         'creation_time': db_url_mapping.creation_time,
         'expiration_time': db_url_mapping.expiration_time,
         'is_active': db_url_mapping.is_active,
         'short_url': db_url_mapping.short_url,
         'long_url': db_url_mapping.long_url}
    df.append(d)
db.close()
pd.DataFrame(df)

,id,creation_time,expiration_time,is_active,short_url,long_url
0,1,2024-09-27 02:29:13.572413,2024-09-27 02:49:13.572413,True,HVDW4,https://www.keysight.com/us/en/home.html
1,2,2024-09-27 02:37:14.979259,2024-09-27 02:38:14.979259,False,5LT40,https://www.google.com
2,3,2024-09-27 02:37:15.532955,2024-09-27 02:39:15.532955,False,DPJ4S,https://www.bing.com
3,4,2024-09-27 02:37:16.090145,2024-09-27 02:40:16.090145,True,3XM3B,https://www.amazon.com
4,5,2024-09-27 02:37:16.634263,2024-09-27 02:41:16.634263,True,XPHKC,https://github.com/


In [25]:
current_time = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
current_time

datetime.datetime(2024, 9, 27, 2, 40, 7, 146759)

#### After 1 minute, the program automatically deativate expired item [amazon]

In [26]:
db = SessionLocal()
df = []

for db_url_mapping in db.query(URLMapping).all():
    d = {'id': db_url_mapping.id,
         'creation_time': db_url_mapping.creation_time,
         'expiration_time': db_url_mapping.expiration_time,
         'is_active': db_url_mapping.is_active,
         'short_url': db_url_mapping.short_url,
         'long_url': db_url_mapping.long_url}
    df.append(d)
db.close()
pd.DataFrame(df)

,id,creation_time,expiration_time,is_active,short_url,long_url
0,1,2024-09-27 02:29:13.572413,2024-09-27 02:49:13.572413,True,HVDW4,https://www.keysight.com/us/en/home.html
1,2,2024-09-27 02:37:14.979259,2024-09-27 02:38:14.979259,False,5LT40,https://www.google.com
2,3,2024-09-27 02:37:15.532955,2024-09-27 02:39:15.532955,False,DPJ4S,https://www.bing.com
3,4,2024-09-27 02:37:16.090145,2024-09-27 02:40:16.090145,False,3XM3B,https://www.amazon.com
4,5,2024-09-27 02:37:16.634263,2024-09-27 02:41:16.634263,True,XPHKC,https://github.com/


In [27]:
current_time = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
current_time

datetime.datetime(2024, 9, 27, 2, 40, 55, 607110)

#### After 1 minute, the program automatically deativate expired item [github]

In [33]:
db = SessionLocal()
df = []

for db_url_mapping in db.query(URLMapping).all():
    d = {'id': db_url_mapping.id,
         'creation_time': db_url_mapping.creation_time,
         'expiration_time': db_url_mapping.expiration_time,
         'is_active': db_url_mapping.is_active,
         'short_url': db_url_mapping.short_url,
         'long_url': db_url_mapping.long_url}
    df.append(d)
db.close()
pd.DataFrame(df)

,id,creation_time,expiration_time,is_active,short_url,long_url
0,1,2024-09-27 02:29:13.572413,2024-09-27 02:49:13.572413,True,HVDW4,https://www.keysight.com/us/en/home.html
1,2,2024-09-27 02:37:14.979259,2024-09-27 02:38:14.979259,False,5LT40,https://www.google.com
2,3,2024-09-27 02:37:15.532955,2024-09-27 02:39:15.532955,False,DPJ4S,https://www.bing.com
3,4,2024-09-27 02:37:16.090145,2024-09-27 02:40:16.090145,False,3XM3B,https://www.amazon.com
4,5,2024-09-27 02:37:16.634263,2024-09-27 02:41:16.634263,False,XPHKC,https://github.com/


In [34]:
current_time = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
current_time

datetime.datetime(2024, 9, 27, 2, 43, 25, 71156)